<a href="https://colab.research.google.com/github/NirjharDebnath/Machine-Learning/blob/main/Satellite_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: unzip a file in sample_data folder

import zipfile
import os

# Define the path to the zip file
zip_file_path = '/content/sample_data/EuroSAT_RGB.zip'

# Define the path to the directory where the files will be extracted
extract_dir = '/content/sample_data/'

# Check if the zip file exists
if os.path.exists(zip_file_path):
  # Create a ZipFile Object
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of zip file in different directory
    zip_ref.extractall(extract_dir)
  print(f"Successfully extracted {zip_file_path} to {extract_dir}")
else:
  print(f"Error: {zip_file_path} not found.")


Error: /content/sample_data/EuroSAT_RGB.zip not found.


In [3]:
! pip install keras


In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
print(tf.__version__)


2.17.1


In [5]:
train_dir = '/content/sample_data/EuroSAT_RGB/'
test_dir = '/content/sample_data/EuroSAT_RGB/'

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [7]:
train_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
test_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/EuroSAT_RGB/'

In [ ]:
train_data.num_classes, train_data.class_indices

In [ ]:
class_names = list(train_data.class_indices.keys())
print(class_names)

## Data Visualization


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for images, labels in train_data:
    plt.imshow(images[0])
    plt.title(class_names[np.argmax(labels[0])])
    break

## Model

In [ ]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy()]
)

In [ ]:
model.fit(train_data, epochs=10, validation_data=test_data)

## Fine Tuning

In [ ]:
base_model.trainable = True
model.fit(
    train_data,
    epochs=15,
    validation_data=test_data
)